In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the websites
URL_LIST = [
    "https://www.redbus.in/online-booking/hrtc",
    "https://www.redbus.in/online-booking/kaac-transport",
    "https://www.redbus.in/online-booking/meghalaya-transport-corporation-mtc",
    "https://www.redbus.in/online-booking/sikkim-nationalised-transport-snt",
    "https://www.redbus.in/online-booking/astc",
    "https://www.redbus.in/online-booking/gsrtc",
    "https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc",
    "https://www.redbus.in/online-booking/ksrtc-kerala",
    "https://www.redbus.in/online-booking/west-bengal-transport-corporation",
    "https://www.redbus.in/online-booking/tsrtc"
]

# Function to initialize and close the Selenium driver
def initialize_driver():
    try:
        driver = webdriver.Chrome()
        driver.maximize_window()
        return driver
    except Exception as e:
        print(f"Error initializing the driver: {str(e)}")
        return None

def close_driver(driver):
    if driver:
        driver.quit()

# Function to load a page
def load_page(driver, url):
    try:
        driver.get(url)
        time.sleep(5)  # Wait for the page to load
    except Exception as e:
        print(f"Error loading page {url}: {str(e)}")

# Function to scrape bus routes
def scrape_bus_routes(driver):
    try:
        route_elements = driver.find_elements(By.CLASS_NAME, 'route')
        bus_routes_link = [route.get_attribute('href') for route in route_elements]
        bus_routes_name = [route.text.strip() for route in route_elements]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {str(e)}")
        return [], []

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load

            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details

        except Exception as e:
            print(f"Error scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error accessing {url}: {str(e)}")
        return []

# Function to find total pages
def find_total_pages(driver):
    try:
        pagination_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'DC_117_paginationTable')]//div[contains(@class, 'DC_117_pageTabs')]")
        if pagination_elements:
            total_pages = int(pagination_elements[-1].text)  # Convert the last pagination number to an integer
            return total_pages
        else:
            return 1  # If no pagination is found, assume only one page
    except Exception as e:
        print(f"Error finding total pages: {str(e)}")
        return 1  # Default to 1 page if unable to determine

# Function to scrape all pages for a URL
def scrape_all_pages_for_url(driver, url):
    all_bus_details = []
    load_page(driver, url)
    pages = find_total_pages(driver)
    print(f"Total pages found: {pages}")
    for page in range(1, pages + 1):
        if page > 1:
            try:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            except Exception as e:
                print(f"Error navigating to page {page}: {str(e)}")
                continue  # Skip to the next page if there's an error
            
        all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
        for link, name in zip(all_bus_routes_link, all_bus_routes_name):
            bus_details = scrape_bus_details(driver, link, name)
            if bus_details:
                all_bus_details.extend(bus_details)
    return all_bus_details

# Function to scrape multiple URLs and save to CSV
def scrape_multiple_urls_and_save():
    driver = initialize_driver()
    if driver:
        for i, url in enumerate(URL_LIST):
            print(f"Scraping URL {i+1}/{len(URL_LIST)}: {url}")
            
            # Scrape bus details for the current URL
            all_bus_details = scrape_all_pages_for_url(driver, url)
            
            # Convert the list of dictionaries to a DataFrame
            df = pd.DataFrame(all_bus_details)
            
            # Save the DataFrame to a CSV file, with a unique name for each URL
            filename = f'Red_bus_csv_{i+1}.csv'
            df.to_csv(filename, index=False)
            
            print(f"Saved bus details to {filename}")
        
        close_driver(driver)

# Start the scraping process for all URLs
scrape_multiple_urls_and_save()


Scraping URL 1/10: https://www.redbus.in/online-booking/hrtc
Total pages found: 4
Error navigating to page 2: Message: 
Stacktrace:
0   chromedriver                        0x0000000104980248 cxxbridge1$str$ptr + 1907280
1   chromedriver                        0x0000000104978730 cxxbridge1$str$ptr + 1875768
2   chromedriver                        0x000000010458c260 cxxbridge1$string$len + 89488
3   chromedriver                        0x00000001045d050c cxxbridge1$string$len + 368700
4   chromedriver                        0x000000010460a7d0 cxxbridge1$string$len + 606976
5   chromedriver                        0x00000001045c512c cxxbridge1$string$len + 322652
6   chromedriver                        0x00000001045c5d7c cxxbridge1$string$len + 325804
7   chromedriver                        0x00000001049484d8 cxxbridge1$str$ptr + 1678560
8   chromedriver                        0x000000010494ce40 cxxbridge1$str$ptr + 1697352
9   chromedriver                        0x000000010492d5ec cxxbridg